In [31]:
#import statements
import astropy as ap
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from astropy.constants import G, k_B, h, c, M_sun, L_sun

ALL UNITS IN CGS

In [32]:
#define constants in CGS

G = G.cgs
k_B = k_B.cgs
h = h.cgs
c = c.cgs
M_sun = M_sun.cgs
L_sun = L_sun.cgs


In [ ]:
#Set up Polynomials for Outer Region

"""
M #we want this to be the black hole mass

"""
def Accretion_Polynomials():
    
    L_solar
    L_
    L_edd = M #assumes hydrogen plasma
    m = M/M_sun
    mdot = Mdot*(c**2)/L_edd
    r_bh = r*(c**2)/(G*M)
    
    def __init__(self):
        self.F = (7e26)*(mdot/m)*(r_bh**-3)*(B**-1)*(C**-0.5)*(Q)
        self.Sigma = (4e5)*(alpha**(-0.1)*m**(0.2)*mdot**(0.7))*(r**(-3/4))*A**(0.1)*B**(-4/5)*C**(1/2)*D**(-17/20)*E**(-1/20)*Q**(7/10)
        self.H = (4e2)*(alpha**(-0.1)*m**(18/20)*mdot**(3/20))*


In [33]:
#Initialize input arrays
resolution = 1000 #number of steps over the radius
radius = np.arange(resolution)
